In [63]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime
import time
import numpy as np
import pandas as pd
import os

# from matplotlib import pyplot as plt
# import plotly.express as px
# %matplotlib inline

In [119]:
def create_year_halfs(years: list):
    '''Обычный ленивый скрипт для создания дат по формату американского времени для гугл трендов'''
    
    dates = []
    quarters = ['-01-01', '-06-30', '-07-01', '-12-31']
    
    for year in sorted(years):
        quarters_year = [str(year) + date for date in quarters]
        dates.extend(quarters_year)
    
    # например, если год 2007, то вернет список ['2007-01-01', '2007-06-30', '2007-07-01', '2007-12-31']
    return dates


def download_csv(link, sleep_time=15, wait_button_time=120):
    '''Используем билиотеку selenium для автоматического открытия заданной ссылки
       Гугл трендов, робот нажимает на кнопку скачать и закрывает браузер'''
    
    
    # открываем драйвер гугла в этой же директории
    driver = webdriver.Chrome(executable_path='./chromedriver')
    time.sleep(5)
    
    # открываем гугл
    driver.get('https://www.google.ru')
    time.sleep(sleep_time)
    
    # открываем ссылку гугл трендов по нашему запросу в гугле
    driver.get(link)
    time.sleep(sleep_time)
    
    # ждем, пока кнопку можно будет нажать
    button = WebDriverWait(driver, wait_button_time).until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.widget-actions-item.export')))
    
    # скачиваем файл
    button.click()
    time.sleep(5)
    
    # закрываем браузер
    driver.close()
    pass


def create_excel_region(ural_regions: dict, years: list, queries: list, path='/Users/ska19/Downloads'):
    ''' Принимает на вход словарь ural_regions: ключ название региона и его код в виде RU-XXX, 
        который будет вставляться в ссылку.
        Затем список годов years, которые Вам нужны для подкачки.
        И последне queries - слова, по которым вы хотите посмотреть поисковую активность.
        path - ваш путь к папке, куда по умолчанию скачиваются файлы.
        Данный скрипт скачивает ежедневные данные с сайта Гугл тренды, собирает их в датафрейм и затем сохраняет
        в виде эксель файла.'''
    

    dates = create_year_halfs(years) # создаем кварталы
    today = pd.Timestamp(datetime.today().date()) # дата на сегодня
    
    for key, value in ural_regions.items():  # идем по каждому региону
        df_region = pd.DataFrame()
        
        for i in range(0, len(queries), 5): # создаем запрос из 5 слов, это максимум для гугл трендов
            words = queries[i:i+5] # берем 5 слов
            query = ','.join(words)
            df_query = pd.DataFrame() # сюда заносим вывод на конкретный регион и запрос по кварталам
            
            
            for j in range(0, len(dates), 2):  # идем по конкретному кварталу
                # нам нужно, что бы новый квартал не выходил за рамки сегодняшней даты
                if today < pd.to_datetime(dates[j], yearfirst=True):
                    print(f"Date {dates[j]} is the future, today is {today}")
                    continue
                    
                # скачиваем файл по ссылке   
                link_to_open = f'https://trends.google.ru/trends/explore?date={dates[j]}%20{dates[j+1]}&geo={value}&q={query}'
                download_csv(link_to_open)
                
                # теперь работаем с самим файлом
                df_quarter = pd.read_csv(path + '/multiTimeline.csv', header=1) # открываем скачанный файл
                df_query = pd.concat([df_query, df_quarter], axis=0) # накладывает фремы друг на друга
                os.remove(path + '/multiTimeline.csv') # удаляем файл, чтобы программа могла прочитать следующий при подкачке
                
            df_query.set_index('День', inplace=True)
            df_query.index = pd.to_datetime(df_query.index, yearfirst=True)
            df_region.index = df_query.index # для метода join нужно, что бы индексы совпадали
            df_region = df_region.join(df_query) # добавляем новые запросы к старым по индексу (индексы должны совпадать XD)
        
        # df_region.to_csv(f'{key}_trends.csv') # можете сохранять в виде csv, если хотите
        df_region.to_excel(f'{key}_trends.xlsx') # сохраняем тренды региона в эксельку
        pass

In [123]:
years = list(range(2019, 2023)) 

ural_regions = {'Свердловская область': 'RU-SVE', 'Курганская область': 'RU-KGN', 'Тюменская область': 'RU-TYU', 
                'Челябинская область': 'RU-CHE', 'Республика Башкортостан':'RU-BA', 
                'Пермский край': 'RU-PER', 'Оренбургская область': 'RU-ORE'}

queries = ['работа', 'кредит', 'мама', 'банкротство', 'депозит', 'мемы', 'вк']

In [124]:
create_excel_region(ural_regions, years, queries)

C:\Users\ska19\AppData\Local\Temp/ipykernel_7592/3253043958.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')


Date 2022-07-01 is the future, today is 2022-06-07 00:00:00
Date 2022-07-01 is the future, today is 2022-06-07 00:00:00


C:\Users\ska19\AppData\Local\Temp/ipykernel_7592/3253043958.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')


KeyboardInterrupt: 